In [1]:
from pathlib import Path
import pandas as pd

df = pd.read_pickle(Path.cwd() / "pickles" /"parsed.pickle")

Hello

In [ ]:
## Get column names
df.columns
# initialise dictionary
adj_dict = {}
# Loops thru each row in the df and gets the spacy doc
for index, row in df.iterrows():
    print("-"*30)
    print(row['title'])
    doc = row['parsed']
    # Loop thru each word of each spacy doc
    for token in doc:
        if token.pos_ == "ADJ":
            adj = token.lemma_
            adj_dict[adj] = adj_dict.get(adj, 0) + 1

# Convert dictionary to lsit of tuples
adj_list = list(adj_dict.items())


print(adj_list)

In [ ]:
import spacy

nlp = spacy.load("en_core_web_sm")

# Create fake data to test function
data = {'title': ['novel1', 'novel2', 'novel3'],
        'text': ['there was a pink happy cat',
                 'the broccoli was soft green and hot',
                 'the athlete went running']}

df = pd.DataFrame(df)
# Add parsed column 
parsed_list = []
for index, row in df.iterrows():
    parsed_obj = nlp(row['text'])
    parsed_list.append(parsed_obj)
    
df['parsed'] = parsed_list

In [ ]:
# Test subject_by_verb_count
import PartOne as po

df_mini = df.iloc[[0]]
#print(df_mini)
out = po.subjects_by_verb_count(df, 'to hear')

for title, pair in out.items():
    print(f"{title}:")
    if pair:
        for pair_dict in pair:
            print(f"\t{pair_dict}")
        else:
            print("\tEmpty")

AttributeError: 'list' object has no attribute 'items'